In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
returns_df = pd.read_csv("/content/lstm_test_returns.csv")

In [ ]:
# Define the objective function for the portfolio optimization
def objective(x,mean_returns,cov_matrix):
    port_returns = np.sum(mean_returns * x)
    port_volatility = np.sqrt(np.dot(x.T, np.dot(cov_matrix, x)))
    penalty = 10 * np.abs(np.sum(x) - 1) # penalty for sum of portfolio weights not equal to 1
    return -port_returns/port_volatility + penalty

In [ ]:
def abc_optimization(n_solutions, n_iter, lb, ub, objective_func,mean_returns,cov_matrix):
    n_dim = len(lb)
    best_sol = None
    best_obj = np.inf
    solutions = np.random.dirichlet(np.ones(len(lb)), size=n_solutions) * (ub-lb) + lb
    solutions = solutions / solutions.sum(axis=1, keepdims=True)
    for i in range(n_solutions):
        sol = np.random.uniform(lb, ub, n_dim)
        obj = objective_func(sol,mean_returns,cov_matrix)
        if obj < best_obj:
            best_sol = sol
            best_obj = obj
        for j in range(n_iter):
            partner = np.random.choice(list(set(range(n_solutions)) - set([i])))
            phi = np.random.uniform(-1, 1, n_dim)
            new_sol = sol + phi * (sol - solutions[partner])
            new_sol = np.maximum(new_sol, lb)
            new_sol = np.minimum(new_sol, ub)
            new_obj = objective_func(new_sol,mean_returns,cov_matrix)
            if new_obj < obj:
                sol = new_sol
                obj = new_obj
                if obj < best_obj:
                    best_sol = sol
                    best_obj = obj
    return best_sol, best_obj

In [ ]:
# Define the hybrid algorithm
def hybrid_algorithm(n_solutions, n_iter, lb, ub, objective_func, alpha, beta0, gamma,mean_returns,cov_matrix):
    abc_solutions, abc_obj = abc_optimization(n_solutions, n_iter, lb, ub, objective_func,mean_returns,cov_matrix)
    return abc_solutions, abc_obj


In [ ]:
returns_df

,PFE,JNJ,GOOGL,AAPL,COST,WMT,KR,JPM,BAC,HSBC
0,0.009278,0.023532,-0.001317,-0.002487,0.109745,0.009333,-0.000589,0.036249,0.013793,0.003713
1,0.006646,0.018055,-0.004994,-0.005049,0.091762,0.007514,-0.005134,0.028507,-0.000543,-0.000154
2,0.009683,0.018818,-0.007247,-0.007777,0.097707,0.007716,0.003025,0.029644,0.021876,0.009885
3,0.007665,0.012438,-0.009241,-0.009711,0.073133,0.005877,-0.000371,0.022066,0.012102,0.006415
4,0.007187,0.008417,-0.009631,-0.010794,0.057347,0.004635,0.000343,0.017417,0.013653,0.007042
...,...,...,...,...,...,...,...,...,...,...
199,0.019016,-0.002627,0.015054,0.011906,-0.013947,0.014308,0.001806,0.025987,0.032756,0.008312
200,0.019883,-0.000072,0.016500,0.013944,-0.012368,0.015091,0.002858,0.028664,0.035819,0.009934
201,0.019210,0.001887,0.016783,0.014978,-0.010141,0.014775,0.003375,0.029270,0.035930,0.010328
202,0.018323,0.003882,0.017034,0.015898,-0.007939,0.014268,0.003861,0.029344,0.035713,0.010513


In [ ]:
roi=[]
s_r=[]
assets = ["PFE", "JNJ", "GOOGL", "AAPL", "COST", "WMT", "KR", "JPM",
          "BAC", "HSBC"]

In [ ]:
n_solutions = 100
n_iter = 50
alpha = 0.1
beta0 = 1
gamma = 2

lb = np.array([0] * len(assets))
ub = np.array([1] * len(assets))

In [ ]:
lb

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
ub

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
returns_df

,PFE,JNJ,GOOGL,AAPL,COST,WMT,KR,JPM,BAC,HSBC
0,0.009278,0.023532,-0.001317,-0.002487,0.109745,0.009333,-0.000589,0.036249,0.013793,0.003713
1,0.006646,0.018055,-0.004994,-0.005049,0.091762,0.007514,-0.005134,0.028507,-0.000543,-0.000154
2,0.009683,0.018818,-0.007247,-0.007777,0.097707,0.007716,0.003025,0.029644,0.021876,0.009885
3,0.007665,0.012438,-0.009241,-0.009711,0.073133,0.005877,-0.000371,0.022066,0.012102,0.006415
4,0.007187,0.008417,-0.009631,-0.010794,0.057347,0.004635,0.000343,0.017417,0.013653,0.007042
...,...,...,...,...,...,...,...,...,...,...
199,0.019016,-0.002627,0.015054,0.011906,-0.013947,0.014308,0.001806,0.025987,0.032756,0.008312
200,0.019883,-0.000072,0.016500,0.013944,-0.012368,0.015091,0.002858,0.028664,0.035819,0.009934
201,0.019210,0.001887,0.016783,0.014978,-0.010141,0.014775,0.003375,0.029270,0.035930,0.010328
202,0.018323,0.003882,0.017034,0.015898,-0.007939,0.014268,0.003861,0.029344,0.035713,0.010513


In [ ]:
returns_df.shape[0]

204

In [ ]:
def opt_portfolio(i,mean_returns,cov_matrix):
    optimized_portfolio = hybrid_algorithm(n_solutions, n_iter, lb, ub, objective, alpha, beta0, gamma,mean_returns,cov_matrix)
    print("Portfolio allocation:", optimized_portfolio[0])
    portfolio_returns = np.sum(optimized_portfolio[0] * mean_returns)
    portfolio_volatility = np.sqrt(np.dot(optimized_portfolio[0].T, np.dot(cov_matrix,optimized_portfolio[0])))
    sharpe_ratio = portfolio_returns / portfolio_volatility
    sortino_ratio = portfolio_returns / np.sqrt(np.mean(np.minimum(returns_df, 0)**2))
    max_drawdown = np.max(np.maximum.accumulate((1+returns_df).cumprod()) - (1+returns_df).cumprod())
    print("Results for day: ",i)
    print("Portfolio allocation:\n",optimized_portfolio[0])
    print("Sharpe ratio:\n", sharpe_ratio)
    print("Sortino ratio:\n", sortino_ratio)
    print("Max drawdown:\n", max_drawdown)
    return(optimized_portfolio[0])

In [ ]:
def add(L):
  res = list()
  for j in range(0, len(L[0])):
    tmp = 0
    for i in range(0, len(L)):
        tmp = tmp + L[i][j]
    res.append(tmp)
  arr = np.array(res)
  result = arr/len(L[0])
  res = result.tolist()
  return(res)

In [ ]:
def calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov):
  cumulative_returns = 0
  cumulative_volatilities = 0

  for i in range(len(normalized_opt)):
      #print(i)
      # Calculate portfolio returns

      portfolio_returns = np.sum(normalized_opt[i] * interval_returns[i])
      #print(portfolio_returns)
      # Calculate portfolio volatility
      portfolio_volatility = np.sqrt(np.dot(normalized_opt[i].T, np.dot(interval_cov[i], normalized_opt[i])))
      #print(portfolio_volatility)
      # Add to cumulative returns and volatilities
      cumulative_returns += portfolio_returns
      cumulative_volatilities += portfolio_volatility

  sharpe_ratio = cumulative_returns / cumulative_volatilities
  print("Sharpe Ratio:", sharpe_ratio)
  return(sharpe_ratio)


In [ ]:
def calculate_roi(normalized_opt,ditv):
  i=0
  k=1
  capital=10000
  final_investment=[]
  final_amount=0
  invest = np.array([capital] * len(assets))
  investment = [x * y for x, y in zip(invest, normalized_opt[0])]
  remain=10000-sum(investment)
  for j in range(1,151):
    row_list = returns_df.loc[j, :].values.flatten().tolist()
    investment = [(x+(x * y)) for x, y in zip(investment, row_list)]
    i=i+1
    if(i==ditv and j!=150 ):
      print(investment)
      amount=sum(investment)+remain
      print("Investment at day ",j," is ",amount)
      cap = sum(investment)+remain
      investment = np.array([cap] * len(assets))
      investment = [x * y for x, y in zip(investment, normalized_opt[k])]
      print("New portfolio investment : ",investment)
      temp=sum(investment)
      print("Investment amount ",j," is ",amount)
      remain=cap-sum(investment)
      print("Remaning amount",remain)
      k=k+1
      i=0
    elif(j==150):
      print("Final Portfolio investment")
      final_investment=investment.copy()
      print(investment)
      amount=sum(investment)+remain
      final_amount=amount
      print("Investment at day ",j," is ",amount)

  n_periods = 252 / 150
  roi = ((final_amount / capital) - 1) * 100
  annualized_roi = (((1 + roi / 100) ** n_periods) - 1)*100
  print("Annualized ROI:", annualized_roi , "%")
  return(annualized_roi)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==5):
    temp_df=returns_df.iloc[j-4:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt",len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,5))

        PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
1  0.006646  0.018055 -0.004994 -0.005049  0.091762  0.007514 -0.005134   
2  0.009683  0.018818 -0.007247 -0.007777  0.097707  0.007716  0.003025   
3  0.007665  0.012438 -0.009241 -0.009711  0.073133  0.005877 -0.000371   
4  0.007187  0.008417 -0.009631 -0.010794  0.057347  0.004635  0.000343   
5  0.005267  0.002656 -0.010171 -0.011635  0.036429  0.002837 -0.002752   

        JPM       BAC      HSBC  
1  0.028507 -0.000543 -0.000154  
2  0.029644  0.021876  0.009885  
3  0.022066  0.012102  0.006415  
4  0.017417  0.013653  0.007042  
5  0.010648  0.004525  0.002903  
                PFE       JNJ         GOOGL      AAPL      COST       WMT  \
PFE    2.596080e-06  0.000008  9.907360e-07  0.000001  0.000031  0.000002   
JNJ    7.963952e-06  0.000046  1.225625e-05  0.000016  0.000170  0.000014   
GOOGL  9.907360e-07  0.000012  4.546701e-06  0.000006  0.000044  0.000004   
AAPL   1.408187e-06  0.000016  5.5833

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.49481782 0.07010819 0.00398397 0.03331268 0.13365241 0.00162402
 0.00205011 0.06291354 0.11354364 0.08683761]
Results for day:  5
Portfolio allocation:
 [0.49481782 0.07010819 0.00398397 0.03331268 0.13365241 0.00162402
 0.00205011 0.06291354 0.11354364 0.08683761]
Sharpe ratio:
 0.05992459896400545
Sortino ratio:
 PFE      0.029920
JNJ      0.029499
GOOGL    0.038863
AAPL     0.041246
COST     0.016453
WMT      0.047829
KR       0.055996
JPM      0.017292
BAC      0.013258
HSBC     0.038865
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
11  0.001284 -0.007724 -0.005857 -0.011483 -0.003568 -0.000261 -0.000580   
12  0.001579 -0.006439 -0.005227 -0.010925 -0.001945  0.000442  0.000528   
13  0.001577 -0.003473 -0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.51250535 0.0025008  0.05534943 0.         0.         0.00656668
 0.26060882 0.06234281 0.0469606  0.06605845]
Results for day:  5
Portfolio allocation:
 [0.51250535 0.0025008  0.05534943 0.         0.         0.00656668
 0.26060882 0.06234281 0.0469606  0.06605845]
Sharpe ratio:
 0.442367040875508
Sortino ratio:
 PFE      0.126406
JNJ      0.124629
GOOGL    0.164189
AAPL     0.174259
COST     0.069511
WMT      0.202071
KR       0.236577
JPM      0.073058
BAC      0.056011
HSBC     0.164201
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
16  0.003118  0.007113 -0.001228 -0.005359  0.033228  0.003487  0.009602   
17  0.003518  0.007143 -0.001670 -0.005412  0.031807  0.003833  0.008673   
18  0.004182  0.009732 -0.0

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.         0.04461926 0.         0.04328062 0.32788458 0.49979102
 0.         0.07578013 0.         0.08178991]
Results for day:  5
Portfolio allocation:
 [0.         0.04461926 0.         0.04328062 0.32788458 0.49979102
 0.         0.07578013 0.         0.08178991]
Sharpe ratio:
 7.9147266315416385
Sortino ratio:
 PFE      0.409047
JNJ      0.403297
GOOGL    0.531315
AAPL     0.563900
COST     0.224936
WMT      0.653899
KR       0.765559
JPM      0.236413
BAC      0.181252
HSBC     0.531352
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
146 -0.004021  0.002876  0.003502  0.000546  0.007492  0.002360 -0.000772   
147 -0.005866  0.002509  0.002322  0.000262  0.008200  0.001449 -0.001744   
148 -0.007748  0.001507

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.00000000e+00 0.00000000e+00 6.02889654e-02 1.55269451e-03
 9.03473920e-01 3.53580506e-03 0.00000000e+00 3.05623365e-04
 0.00000000e+00 8.31452767e-03]
Results for day:  5
Portfolio allocation:
 [0.00000000e+00 0.00000000e+00 6.02889654e-02 1.55269451e-03
 9.03473920e-01 3.53580506e-03 0.00000000e+00 3.05623365e-04
 0.00000000e+00 8.31452767e-03]
Sharpe ratio:
 15.194730654233117
Sortino ratio:
 PFE      0.473979
JNJ      0.467317
GOOGL    0.615655
AAPL     0.653414
COST     0.260642
WMT      0.757698
KR       0.887084
JPM      0.273941
BAC      0.210023
HSBC     0.615699
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
Length of opt 30
[[0.319 0.    0.    0.085 0.248 0.032 0.    0.088 0.    0.221]
 [0.492 0.069 0.003 0.032 0.132 0.002 0.002 0.062 0.112 0.086]
 [0.505 0.002 0.054 0.    0. 

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==10):
    temp_df=returns_df.iloc[j-9:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt", len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,10))

         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
1   0.006646  0.018055 -0.004994 -0.005049  0.091762  0.007514 -0.005134   
2   0.009683  0.018818 -0.007247 -0.007777  0.097707  0.007716  0.003025   
3   0.007665  0.012438 -0.009241 -0.009711  0.073133  0.005877 -0.000371   
4   0.007187  0.008417 -0.009631 -0.010794  0.057347  0.004635  0.000343   
5   0.005267  0.002656 -0.010171 -0.011635  0.036429  0.002837 -0.002752   
6   0.004247  0.000023 -0.008946 -0.011212  0.025851  0.001602 -0.001911   
7   0.002394 -0.004605 -0.008891 -0.011675  0.010617  0.000061 -0.004307   
8   0.001759 -0.006825 -0.008151 -0.011874  0.002699 -0.000580 -0.003888   
9   0.000990 -0.008870 -0.007836 -0.012266 -0.004181 -0.001084 -0.004083   
10  0.000877 -0.009821 -0.007422 -0.012565 -0.007826 -0.000992 -0.003402   

         JPM       BAC      HSBC  
1   0.028507 -0.000543 -0.000154  
2   0.029644  0.021876  0.009885  
3   0.022066  0.012102  0.006415  
4   0.017417  0.013653 

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.02076154 0.0020999  0.01834695 0.01195411 0.06198749 0.09996268
 0.00176045 0.01662676 0.         0.75627145]
Results for day:  10
Portfolio allocation:
 [0.02076154 0.0020999  0.01834695 0.01195411 0.06198749 0.09996268
 0.00176045 0.01662676 0.         0.75627145]
Sharpe ratio:
 1.4411812790209901
Sortino ratio:
 PFE      1.028235
JNJ      1.013783
GOOGL    1.335585
AAPL     1.417497
COST     0.565429
WMT      1.643728
KR       1.924414
JPM      0.594281
BAC      0.455619
HSBC     1.335679
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
21  0.005329  0.013996  0.002160  0.000377  0.039615  0.005220  0.009950   
22  0.005670  0.013685  0.001995  0.000619  0.036887  0.005127  0.008873   
23  0.005762  0.012708  0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.88850055 0.         0.09478194 0.06321786 0.00193363 0.0848441
 0.         0.         0.01829808 0.00381445]
Results for day:  10
Portfolio allocation:
 [0.88850055 0.         0.09478194 0.06321786 0.00193363 0.0848441
 0.         0.         0.01829808 0.00381445]
Sharpe ratio:
 17.65935498058845
Sortino ratio:
 PFE      0.789725
JNJ      0.778626
GOOGL    1.025782
AAPL     1.088693
COST     0.434272
WMT      1.262448
KR       1.478026
JPM      0.456431
BAC      0.349933
HSBC     1.025854
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
31  0.005631  0.001664 -0.002709 -0.002279  0.001966  0.002188 -0.002849   
32  0.005051  0.001420 -0.002948 -0.001938  0.001610  0.001754 -0.003055   
33  0.004363  0.000959 -0.00

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.44529967 0.03288884 0.00055731 0.00685216 0.27003684 0.00151662
 0.18940552 0.         0.         0.05804349]
Results for day:  10
Portfolio allocation:
 [0.44529967 0.03288884 0.00055731 0.00685216 0.27003684 0.00151662
 0.18940552 0.         0.         0.05804349]
Sharpe ratio:
 6.555967758445678
Sortino ratio:
 PFE      0.423723
JNJ      0.417767
GOOGL    0.550378
AAPL     0.584132
COST     0.233006
WMT      0.677360
KR       0.793026
JPM      0.244895
BAC      0.187755
HSBC     0.550416
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
141  0.002229  0.004476  0.004821 -0.000502  0.008065  0.005745  0.003061   
142  0.001575  0.004430  0.005807  0.000201  0.007858  0.005691  0.002542   
143  0.000600  0.004965

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.         0.17166715 0.         0.         0.79266351 0.01486687
 0.         0.         0.         0.01227907]
Results for day:  10
Portfolio allocation:
 [0.         0.17166715 0.         0.         0.79266351 0.01486687
 0.         0.         0.         0.01227907]
Sharpe ratio:
 6.760508030542915
Sortino ratio:
 PFE      0.960472
JNJ      0.946972
GOOGL    1.247566
AAPL     1.324079
COST     0.528166
WMT      1.535402
KR       1.797589
JPM      0.555116
BAC      0.425592
HSBC     1.247653
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
Length of opt 15
[[0.307 0.    0.    0.002 0.133 0.085 0.242 0.039 0.008 0.175]
 [0.02  0.002 0.018 0.011 0.062 0.1   0.002 0.016 0.    0.763]
 [0.768 0.    0.081 0.054 0.002 0.072 0.    0.    0.015 0.002]
 [0.348 0.124 0.011 0.405 0.    0.011 0.    0.05

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==15):
    temp_df=returns_df.iloc[j-14:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt", len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,15))

         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
1   0.006646  0.018055 -0.004994 -0.005049  0.091762  0.007514 -0.005134   
2   0.009683  0.018818 -0.007247 -0.007777  0.097707  0.007716  0.003025   
3   0.007665  0.012438 -0.009241 -0.009711  0.073133  0.005877 -0.000371   
4   0.007187  0.008417 -0.009631 -0.010794  0.057347  0.004635  0.000343   
5   0.005267  0.002656 -0.010171 -0.011635  0.036429  0.002837 -0.002752   
6   0.004247  0.000023 -0.008946 -0.011212  0.025851  0.001602 -0.001911   
7   0.002394 -0.004605 -0.008891 -0.011675  0.010617  0.000061 -0.004307   
8   0.001759 -0.006825 -0.008151 -0.011874  0.002699 -0.000580 -0.003888   
9   0.000990 -0.008870 -0.007836 -0.012266 -0.004181 -0.001084 -0.004083   
10  0.000877 -0.009821 -0.007422 -0.012565 -0.007826 -0.000992 -0.003402   
11  0.001284 -0.007724 -0.005857 -0.011483 -0.003568 -0.000261 -0.000580   
12  0.001579 -0.006439 -0.005227 -0.010925 -0.001945  0.000442  0.000528   
13  0.001577

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.8339104  0.         0.         0.         0.         0.02621251
 0.08145955 0.         0.07598553 0.00983537]
Results for day:  15
Portfolio allocation:
 [0.8339104  0.         0.         0.         0.         0.02621251
 0.08145955 0.         0.07598553 0.00983537]
Sharpe ratio:
 11.370398460343013
Sortino ratio:
 PFE      1.297481
JNJ      1.279244
GOOGL    1.685310
AAPL     1.788671
COST     0.713488
WMT      2.074142
KR       2.428325
JPM      0.749894
BAC      0.574923
HSBC     1.685429
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
31  0.005631  0.001664 -0.002709 -0.002279  0.001966  0.002188 -0.002849   
32  0.005051  0.001420 -0.002948 -0.001938  0.001610  0.001754 -0.003055   
33  0.004363  0.000959 -0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.24837867 0.10740648 0.00419897 0.31563499 0.00638784 0.03996934
 0.00117919 0.07854466 0.         0.02063982]
Results for day:  15
Portfolio allocation:
 [0.24837867 0.10740648 0.00419897 0.31563499 0.00638784 0.03996934
 0.00117919 0.07854466 0.         0.02063982]
Sharpe ratio:
 15.202086837745599
Sortino ratio:
 PFE      0.286330
JNJ      0.282306
GOOGL    0.371917
AAPL     0.394727
COST     0.157454
WMT      0.457725
KR       0.535887
JPM      0.165488
BAC      0.126875
HSBC     0.371943
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
46 -0.003088  0.003464  0.004932  0.008114  0.002817 -0.000832  0.001401   
47 -0.003310  0.004406  0.006243  0.009110  0.003844 -0.000672  0.002291   
48 -0.003365  0.005049  0

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.16434148 0.01129784 0.         0.         0.45867212 0.13972229
 0.13994078 0.         0.05824755 0.00731071]
Results for day:  15
Portfolio allocation:
 [0.16434148 0.01129784 0.         0.         0.45867212 0.13972229
 0.13994078 0.         0.05824755 0.00731071]
Sharpe ratio:
 7.329143124822649
Sortino ratio:
 PFE      0.824101
JNJ      0.812518
GOOGL    1.070432
AAPL     1.136082
COST     0.453175
WMT      1.317400
KR       1.542362
JPM      0.476299
BAC      0.365165
HSBC     1.070508
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
136  0.002859  0.002184 -0.001467 -0.003229  0.005325  0.002823  0.003146   
137  0.003023  0.002193 -0.000159 -0.002871  0.004520  0.003632  0.003102   
138  0.003134  0.002921

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.         0.06474831 0.         0.         0.81531548 0.01767124
 0.         0.06434709 0.00472548 0.03737886]
Results for day:  15
Portfolio allocation:
 [0.         0.06474831 0.         0.         0.81531548 0.01767124
 0.         0.06434709 0.00472548 0.03737886]
Sharpe ratio:
 6.597082498653399
Sortino ratio:
 PFE      1.328490
JNJ      1.309818
GOOGL    1.725588
AAPL     1.831419
COST     0.730540
WMT      2.123713
KR       2.486361
JPM      0.767816
BAC      0.588664
HSBC     1.725709
dtype: float64
Max drawdown:
 PFE      0.500034
JNJ      0.386320
GOOGL    0.353318
AAPL     0.257108
COST     1.196926
WMT      0.306172
KR       0.248281
JPM      0.823145
BAC      1.156644
HSBC     0.372075
dtype: float64
Length of opt 10
[[0.171 0.082 0.    0.    0.217 0.179 0.    0.025 0.063 0.256]
 [0.811 0.    0.    0.    0.    0.025 0.078 0.    0.073 0.009]
 [0.301 0.13  0.004 0.383 0.007 0.048 0.001 0.095 0.    0.024]
 [0.024 0.    0.607 0.177 0.    0.052 0.128 0.  

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
s_r

[2.5134164232137244, 2.001326201107587, 2.351690821820833]

In [ ]:
roi

[353.3021994818452, 326.42671455804725, 254.17448578080467]